In [40]:
# autoreload

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os

if os.getcwd().split("/")[-1] != "Road-Segmentation-ML":
    os.chdir("..")
print("CWD:", os.getcwd())

CWD: /home/nadezhda/Desktop/Road-Segmentation-ML


In [42]:
import torch
from omegaconf import OmegaConf
import segmentation_models_pytorch as smp
from utils import set_seeds
from train_utils import prepare_data, prepare_model, prepare_optimizer, train

In [43]:
# create folder models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

In [44]:
args = OmegaConf.create(
    dict(
        # General
        seed=0,
        # Data folders
        train_image_folders=["datasets/train/images/", "datasets/massachusetts_384/images/"], # add Massachusetts here
        train_gt_folders=["datasets/train/groundtruth/", "datasets/massachusetts_384/groundtruth/"], # add Massachusetts here
        val_image_folders=["datasets/validation/images/"], 
        val_gt_folders=["datasets/validation/groundtruth/"],
        weighted_random_sampler=False,
        # Data tranforms
        # random_resized_crop: crop a random portion of image and resize it to a given size
        random_resized_crop=False,
        output_size=(400, 400),  # expected output size of the crop, for each edge.
        input_size=(384),  # resize
        random_resized_crop_scale=(0.5, 0.5),
        # random_horizontal_flip: randomly flip the image horizontally with a given probability
        random_horizontal_flip=False,
        # random_vertical_flip: randomly flip the image vertically with a given probability
        random_vertical_flip=False,
        # random_rotation: randomly rotate the image with a given probability
        random_rotation=False,
        degrees=5,  # range of degrees to select from
        # color_jitter: randomly change the brightness, contrast and saturation of an image
        color_jitter=False,
        brightness=0.1,  # how much to jitter brightness.
        contrast=0.1,  # how much to jitter contrast.
        saturation=0.1,  # how much to jitter saturation.
        hue=0.1,  # how much to jitter hue.
        # normalization
        normalization=True,  # TODO: Should it always be True?
        normalization_flag = "A", # "A" for AIcrowd, "AM" for AIcrowd + Massachusetts, "AK" for AIcrowd + Kaggle
        # Data loaders
        batch_size=2,
        train_size=0.8,
        val_size=0.2,
        # Model
        # UNetV1
        model_name="UNetV3",  # Change for v3
        model_save_name="models/checkpoints/unetv3-resnet50-normalization-BEST2.pt",
        # Optimizer
        optim_name="adam",  # sgd
        optim_lr=0.001,
        optim_momentum=0.9,  # TODO: Try with (optim_momentum != 0) and without (momentum = 0)?
        # Training
        n_steps=2000,
        eval_freq=100,
        # Wandb logging
        wandb_project="road-segmentation",
        wandb_run="unet-v3",
        entity="feeit",
    )
)

In [ ]:
# set seeds
set_seeds()

In [45]:
# prepare train and validation loaders
train_loader, val_loader = prepare_data(args)

Using Resize with input size=384.
Using ToTensor.
Using Normalize with mean=[0.358, 0.365, 0.3316] and std=[0.1976, 0.1917, 0.194].
Using WeightedRandomSampler.


/home/nadezhda/.local/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [46]:
# prepare model
model = prepare_model(args)
# define loss function
criterion = torch.nn.BCEWithLogitsLoss()
# criterion = DiceLoss()
# prepare optimizer
optimizer = prepare_optimizer(model, args)

Initializing UNetV3 model.
Initializing Adam optimizer with lr=0.001.


In [47]:
# MODEL_CHECKPOINT = "models/checkpoints/unetv3-resnet50-normalization-BEST.pt"
# model.load_state_dict(torch.load(MODEL_CHECKPOINT))

In [48]:
# device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

UNetV3(
  (model): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace

In [ ]:
# train
trained_model = train(
    model, device, train_loader, val_loader, criterion, optimizer, args
)

Submission

In [ ]:
import torch
import torchvision.transforms.v2 as transforms
import torchvision.models.segmentation as models
import matplotlib.pyplot as plt
from datasets.TestDataset import TestDataset
from models.UNetV3 import UNetV3
from examples.mask_to_submission import *

In [ ]:
# select checkpoint
MODEL = "models/checkpoints/unetv3-resnet50-normalization-BEST2.pt"
# load checkpoint
try:
    checkpoint = torch.load(MODEL)
except:
    print("Loading checkpoint failed. Trying to load it with map_location.")
    checkpoint = torch.load(MODEL, map_location=torch.device("cpu"))
# create model
model = UNetV3()
# load model weights
model.load_state_dict(checkpoint)

In [ ]:
# path to the test folder
test_folder = "datasets/test/"

# set mean and std
mean = [0.3353, 0.3328, 0.2984]
std = [0.1967, 0.1896, 0.1897]

# define transformations
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

# create test dataset
test_dataset = TestDataset(test_folder, transform=transform)

In [ ]:
# set threshold
THR = 0.5
# create folder predictions if it doesn't exist
if not os.path.exists("predictions"):
    os.makedirs("predictions")
# save predictions
prediction_filenames = []
for i in range(len(test_dataset)):
    # get image
    image = test_dataset[i]
    # create prediction
    prediction = model(image.unsqueeze(0))
    # threshold prediction
    prediction = torch.sigmoid(prediction)
    prediction = (prediction > THR).float()
    # save prediction
    prediction_filename = "predictions/prediction_" + str(i + 1) + ".png"
    prediction_filenames.append(prediction_filename)
    plt.imsave(prediction_filename, prediction.squeeze().detach().numpy(), cmap="gray")

In [ ]:
# create submission
masks_to_submission("submission.csv", *prediction_filenames)